# Notebook 6: Advanced Applications and Conclusion

In this final notebook, we build on the foundations from previous notebooks and explore advanced and practical applications of self-supervised learning (SSL). We’ll look at how the learned representations (like those from CLIP or other SSL models) can be applied to real-world scenarios, fine-tuned for specific tasks, or extended to new domains. We will also briefly mention state-of-the-art directions (as of 2025) in SSL.

## 6.1 Fine-Tuning CLIP for Custom Classification

**Scenario:** You have a specific classification task, say classifying bird species from photos. You have a moderate dataset (perhaps a few thousand images labeled with species). Rather than training a CNN from scratch, you want to leverage CLIP’s powerful image encoder.

**Approach:** We use the CLIP image encoder and attach a new classifier head:
- **Option A:** Fine-tune the whole model on the bird dataset.
- **Option B:** Freeze the CLIP backbone and just train the new head (this is faster and works well if data is limited).

Let's do Option B (feature extraction) as an example, as it’s often safer with limited data:

In [ ]:
import torch
import torch.nn as nn

# Assume we have train_loader and val_loader for bird dataset (images, labels)
# We'll use CLIP's image encoder from previous notebook:
image_encoder = model.visual  # this is the image encoder inside CLIPModel
image_encoder.eval()  # we won't train it (freezing)
for param in image_encoder.parameters():
    param.requires_grad = False

# New classifier head
num_classes = 200  # e.g., 200 bird species in CUB-200 dataset
classifier = nn.Linear(image_encoder.embed_dim, num_classes).to(device)

# Optimizer for just the classifier
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

Now training loop (using the frozen backbone):


In [ ]:
for epoch in range(10):
    classifier.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        # Get image features from CLIP encoder
        with torch.no_grad():
            feats = image_encoder(images)  # shape [batch, embed_dim]
        logits = classifier(feats)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    # Validate
    classifier.eval()
    correct = total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            feats = image_encoder(images)
            logits = classifier(feats)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    val_acc = correct / total
    print(f"Epoch {epoch+1}: Train loss {avg_loss:.3f}, Val acc {val_acc:.3%}")

## 6.2 Building an Embedding Search Engine

Representations from SSL models can be indexed in a database for fast similarity search. For example, using CLIP or an SSL-pretrained ResNet (like a MoCo or SimCLR model), you can generate embeddings for your entire image collection. Then:

- To find similar images to a query image, embed the query and do a nearest neighbor search in the database of embeddings.
- To find images matching a text description (if using CLIP), embed the text and do the same.

This is often done with specialized libraries or vector databases (like FAISS, Annoy, Milvus, etc.) that handle large-scale similarity search efficiently. Example with FAISS: (We'll conceptually outline, not actually run heavy code)

In [ ]:
import faiss

# Suppose img_features_array is a NumPy array of shape [N, d] for N images
img_features_array = img_features.cpu().numpy().astype('float32')

# Build FAISS index
index = faiss.IndexFlatIP(image_encoder.embed_dim)  # Inner product (assuming features are normalized)
index.add(img_features_array)

# Query with a new image
query_feat = image_encoder(preprocess(query_image).unsqueeze(0).to(device))
query_feat = query_feat / query_feat.norm()  # normalize
query_feat_np = query_feat.cpu().numpy().astype('float32')
D, I = index.search(query_feat_np, k=5)  # find 5 nearest
print("Top 5 similar image indices:", I[0])

## 6.3 Self-Supervised Learning in Practice

A few points from practical experience:
- **Compute Requirements:** Training large SSL models (like SimCLR on ImageNet or CLIP on 400M data) is resource-intensive (multiple GPUs for many hours or days). However, many pre-trained checkpoints are available publicly. Fine-tuning those or using them as is (like we did with CLIP) is feasible on a single GPU.
- **Smaller-Scale SSL:** If you have a smaller domain dataset and want to do SSL from scratch, methods like BYOL, SimSiam (which don’t need large batches) are popular since they converge with less compute. For example, training SimSiam on CIFAR-10 with one GPU is doable and yields a model that can be fine-tuned well.
- **Hugging Face Models:** The transformers library now includes not just BERT and CLIP, but also vision SSL models like BEiT, MAE, DINO, etc. You can directly use these or fine-tune them via the high-level API.
- **Choosing the Right Model:** For vision tasks, CLIP is great if you care about text alignment or zero-shot. If you only need features for images, a dedicated vision SSL model (like MAE pre-trained on ImageNet) might give slightly better pure vision performance. For example, MAE or SwAV models fine-tuned on ImageNet labels sometimes edge out CLIP fine-tuned on ImageNet because CLIP’s objective is slightly different. So, depending on use-case (do you need the multimodal part?), you might pick one or the other.

## 6.4 Other Advanced SSL Approaches

We focused on contrastive and masked modeling and a bit on multimodal. There are other notable SSL ideas:
- **BYOL (Bootstrap Your Own Latent) and SimSiam:** These are non-contrastive methods that surprisingly learn good features without negative pairs. They use two networks (like teacher and student), where one is a moving average of the other (like MoCo's momentum idea), and train the student to predict the teacher’s representation of the same image under a different augmentation. A stop-gradient is used to avoid collapse.
- **SwAV (Swapped Assignments):** A clustering-based method where the network learns to assign images to prototype clusters; it does a clever "swapped prediction" where two augmentations of the same image should yield the same cluster assignments.
- **Barlow Twins / VICReg:** Methods that add regularization to avoid trivial solutions and make different image views’ embeddings similar (correlation-based objectives).
- **Multi-task SSL:** Combining multiple pretext tasks (e.g., contrastive + rotation prediction, or contrastive + clustering).
- **Self-Supervised Learning for Other Data:** Audio (e.g., Wav2Vec 2.0, contrastive predictive coding), Video (predict future frames, shuffle frames), Reinforcement Learning / Robotics (predict sensor inputs or future states).

## 6.5 Hands-On: Few-Shot with Prompt Tuning (Bonus)

A fun advanced trick: Instead of fine-tuning the whole CLIP, one can fine-tune just the text prompts. For instance, for the bird classification, you could learn a vector (or a small set of tokens) that when appended to bird names yields better performance. This is called prompt tuning or prefix learning. OpenAI’s Guided Language-Image Pre-training (CoOp, CoCoOp) research has done this: they find soft prompts that improve zero-shot classification for specific tasks. Implementing prompt tuning properly is involved (optimizing input embeddings through backprop with CLIP fixed). But conceptually:

In [ ]:
# We create a learnable prompt embedding, e.g., 4 tokens of dimension 512
prompt_embed = torch.randn(1, 4, model.text_projection.shape[1], requires_grad=True, device=device)
# Then for each class name, we would attach this prompt and train it via gradient descent
# to maximize the similarity to class images, etc.
# ... (This is an advanced technique and would require a custom training loop)

## 6.6 Closing Thoughts

Self-supervised learning has become a critical component in modern deep learning pipelines. It enables:
- **Pre-training on vast unlabeled (or weakly labeled) data,** which can then be fine-tuned for high performance with fewer labels.
- **Models that generalize to tasks beyond their training,** e.g., CLIP's zero-shot abilities, or BERT being fine-tunable to many NLP tasks.
- **Continual learning:** One can keep updating a model with new unlabeled data to refine its representations without requiring annotations.

It’s also a stepping stone toward more general AI systems. For example, one can combine multiple SSL pre-trained models: imagine a system that has a vision module (like CLIP’s image encoder), a language module (like GPT pre-trained), and maybe an audio module (like HuBERT for speech) – all learned largely from raw data. These modules could be combined (via techniques like model fusion or a multimodal transformer) to create richer, more versatile AI (some research is heading in this direction, e.g., Meta’s ImageBind which aligns multiple modalities in one space).

As of 2025, we see self-supervision:
- In large language models (LLMs) underpinning chatbots and GPT-like systems.
- In computer vision models that serve as backbones for detection/segmentation.
- In recommendation systems or other domains.
- Emerging in scientific domains: SSL on protein sequences, molecules, astronomical data, etc.

What’s next? Areas like self-supervised reinforcement learning (learning policies or state representations without rewards) are being explored. Also, combining SSL with causal representation learning or incorporating domain knowledge (like physics constraints) is an ongoing research.

**Bonus: Using SSL Representations for Novel Tasks**

Let's do one more interactive use-case: image style retrieval. Suppose we have a set of paintings and photos, and we want to retrieve images by style (photograph vs painting). CLIP's embedding space encodes style as well (because style words in text can describe images). We can attempt:

- Take some art images and some photographs.
- Use text queries like "a painting" vs "a photograph".
- See which images align with which.

While we can't run an actual visual output here, one could do:

In [ ]:
texts = ["a painting", "a photograph"]
text_feats = model.get_text_features(**processor(text=texts, return_tensors="pt", padding=True).to(device))
text_feats = text_feats / text_feats.norm(dim=1, keepdim=True)
image_feats = model.get_image_features(**processor(images=images, return_tensors="pt", padding=True).to(device))
image_feats = image_feats / image_feats.norm(dim=1, keepdim=True)
sims = image_feats @ text_feats.T  # shape [num_images, 2]
# Then one could compare similarities to see which images align with which query

### Final Words

This course has taken us through the journey of self-supervised learning:
- We started with the motivation to use unlabeled data and examples like autoencoders.
- We delved into contrastive learning (SimCLR, MoCo) which shaped much of modern SSL.
- Explored classic pretext tasks (rotations, jigsaw, colorization) that paved the way.
- Investigated masked modeling (BERT for NLP, MAE for vision), a dominant force today.
- Learned about CLIP and multimodal learning, extending SSL across modalities.
- Finally, we saw how to apply these learned features in practice, through fine-tuning, search, and more.

Self-supervised learning is a rapidly evolving field. Stay curious: new methods (e.g., aligning many modalities, or SSL that also handles reasoning, etc.) are on the horizon. But the core principle remains – utilize the inherent structure in data to supervise the model. This principle has led to robust and versatile models that are transforming AI.

Thank you for following along this journey! Now, let's put these ideas into practice with projects or experiments of your own.

**Bonus Exercise:** Consider a domain or data type you're interested in (could be medical images, speech signals, sensor data from IoT, etc.). Propose a self-supervised task for that domain. What could be the "missing piece" that the model has to predict? How would you implement and evaluate it? (This is an open-ended exercise, but thinking this way will help in applying SSL beyond the examples we covered.)

## References
- **Grill, J.-B. et al. (2020).** "Bootstrap Your Own Latent: A New Approach to Self-Supervised Learning." NeurIPS. – BYOL, a pioneering negative-free SSL method.
- **Chen, X. & He, K. (2021).** "Exploring Simple Siamese Representation Learning." CVPR. – SimSiam, simpler implementation of BYOL ideas.
- **Zbontar, J. et al. (2021).** "Barlow Twins: Self-Supervised Learning via Redundancy Reduction." ICML.
- **Caron, M. et al. (2020).** "Unsupervised Learning of Visual Features by Contrasting Cluster Assignments." NeurIPS. – SwAV (clustering + contrastive).
- **Assran, M. et al. (2022).** "Masked Siamese Networks for Label-Efficient Learning." CVPR. – Combines masking and Siamese (momentum encoder) concepts.
- **Bommasani, R. et al. (2022).** "Foundation Models in AI." – A report discussing how large pre-trained models (many self-supervised) are becoming foundations for many applications.
- **Goyal, P. et al. (2019).** "Scaling and Benchmarking Self-Supervised Visual Representation Learning." – A study on how different SSL methods perform with varying amounts of data/compute.
- **Trinh, T. et al. (2019).** "Selfie: Self-supervised Pretraining for Image Embedding." – Uses pretext of jigsaw with relative patch location prediction (kind of an inpainting variation).
- **Hugging Face model hub** – Contains many pre-trained SSL models (just search for keywords like "SimCLR", "MAE", "SwAV", etc.) you can experiment with.
- **PyTorch Lightning Bolts** – had some reference implementations of SSL methods (might be useful if you plan to implement one yourself).